In [2]:
from openai import OpenAI
import yaml
import os

# Automatically locate the file in the current directory
CONFIG_PATH = "C:\\Users\\HOME\\github\\config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

api_key = config["api_key"]
base_url = config["base_url"]

client = OpenAI(
    api_key=api_key,  # Replace with your actual API key
    base_url=base_url
)
print("Config loaded")


Config loaded


In [1]:
response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "Give json output like column name, data type and min and max length"},
                {"role": "user", "content": "Generate a dataset schema with 5 columns for transaction data. Take the best transaction dataset from kaggle"}
            ]
        )
print(response.choices[0].message.content)

NameError: name 'client' is not defined

In [6]:
from openai import OpenAI
from schema_agent import SchemaAgent
from schema_models import SchemaPrompt
# Instantiate agent
schema_agent = SchemaAgent(client)

# Define prompt
prompt = SchemaPrompt(
    use_case="Transaction fraud detection",
    prompt='''Generate a User info dataset 
    schema similar to top User info datasets on Kaggle.Creating this.
      Include fields like user name, login time, session time. Need this data set to train a fraud user detection ML model. Decide yourself'''
)

# Get structured output
schema = schema_agent.generate_from_prompt(prompt)

# Print result
print(schema.model_dump_json(indent=2))

{
  "use_case": "Transaction fraud detection",
  "columns": [
    {
      "name": "user_id",
      "type": "int",
      "min": 1.0,
      "max": 1000000.0,
      "format": null,
      "values": null
    },
    {
      "name": "user_name",
      "type": "string",
      "min": null,
      "max": null,
      "format": null,
      "values": null
    },
    {
      "name": "login_time",
      "type": "datetime",
      "min": null,
      "max": null,
      "format": "%Y-%m-%d %H:%M:%S",
      "values": null
    },
    {
      "name": "logout_time",
      "type": "datetime",
      "min": null,
      "max": null,
      "format": "%Y-%m-%d %H:%M:%S",
      "values": null
    },
    {
      "name": "session_duration_seconds",
      "type": "int",
      "min": 1.0,
      "max": 86400.0,
      "format": null,
      "values": null
    },
    {
      "name": "ip_address",
      "type": "string",
      "min": null,
      "max": null,
      "format": null,
      "values": null
    },
    {
      "name

In [15]:
from openai import OpenAI
from schema_agent import SchemaAgent
from schema_models import SchemaPrompt
schema_agent = SchemaAgent(client)
csv_schemaprompt = SchemaPrompt(
    use_case = "Iris dataset transaction",
    csv_path = "C:\\Users\\HOME\\Downloads\\archive\\iris.csv"
)
csv_schema = schema_agent.generate_from_csv(csv_schemaprompt)
print(csv_schema.model_dump_json(indent=2))

{
  "use_case": "Iris dataset transaction",
  "columns": [
    {
      "name": "Id",
      "type": "int",
      "min": 1.0,
      "max": 150.0,
      "format": null,
      "values": null
    },
    {
      "name": "SepalLengthCm",
      "type": "float",
      "min": 4.3,
      "max": 7.9,
      "format": null,
      "values": null
    },
    {
      "name": "SepalWidthCm",
      "type": "float",
      "min": 2.0,
      "max": 4.4,
      "format": null,
      "values": null
    },
    {
      "name": "PetalLengthCm",
      "type": "float",
      "min": 1.0,
      "max": 6.9,
      "format": null,
      "values": null
    },
    {
      "name": "PetalWidthCm",
      "type": "float",
      "min": 0.1,
      "max": 2.5,
      "format": null,
      "values": null
    },
    {
      "name": "Species",
      "type": "categorical",
      "min": null,
      "max": null,
      "format": null,
      "values": [
        "Iris-setosa",
        "Iris-versicolor",
        "Iris-virginica"
      ]
   